
code to create graph

In [ ]:
import json
import pickle
from pathlib import Path
import networkx as nx
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from geopy.distance import distance
import plotly.graph_objects as go
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

buffer_size = 300
n_closest = 3
path_root_city = Path('/content')
filename = 'data_high1.csv'
filepath = path_root_city / filename
filepath_subway = path_root_city / 'метро.geojson'
filepath_bus = path_root_city / 'остановки автобусов.geojson'
filepath_tram = path_root_city / 'остановки трамвай.geojson'
df_squares = gpd.read_file(path_root_city / 'squares1.csv')
df_squares['building_id'] = df_squares['building_id'].astype(int)

def read_geojson(path: Path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def extract_stops(path: Path) -> list[tuple[int, float, float]]:
    data = read_geojson(path)
    coords = list(map(lambda v: (int(v['properties']['osm_id']),
                                 v['geometry']['coordinates'][0],
                                 v['geometry']['coordinates'][1]), data['features']))
    return coords

def draw_plotly(g_):
    import plotly.express as px
    pos_ = nx.get_node_attributes(g_, 'coordinate')
    nodes = list(g_.nodes())
    edge_x = []
    edge_y = []
    edges_text = []
    for edge in g_.edges(data=True):
        x0, y0 = pos_[edge[0]]
        x1, y1 = pos_[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edges_text.extend([f"Meters: {edge[2]['meters']}', f'Meters: {edge[2]['meters']}", None])
    nodes_data = g_.nodes(data=True)
    node_x = []
    node_y = []
    node_text = []
    node_colors = []
    for node in nodes:
        x, y = pos_[node]
        node_x.append(x)
        node_y.append(y)
        attrs = nodes_data[node]
        if attrs['label'] in {'subway', 'bus', 'tram'}:
            node_colors.append('black')
        else:
            node_colors.append('green' if attrs['education'] > 0 else 'red')
        attrs = dict(filter(lambda kv: kv[0] not in {'coordinate', 'label'}, attrs.items()))
        node_text.append('<br>'.join(list(map(lambda kv: f'{kv[0]}: {kv[1]}', attrs.items()))))
    fig = go.Figure()
    df_nodes = pd.DataFrame.from_records(list(dict(nodes_data).values())).drop(columns='coordinate')
    df_nodes['s_cnt'] = df_nodes[['kindergarten', 'bus_buf', 'school', 'tram_stop', 'subway_buf']].sum(axis=1)
    df_nodes['color'] = 'gray'
    df_nodes.loc[df_nodes['s_cnt'] > 0, 'color'] = 'blue'
    df_nodes.loc[df_nodes['education'] == 1, 'color'] = 'green'
    df_nodes.loc[df_nodes['label'].isin({'tram', 'subway', 'bus'}), 'color'] = 'black'
    fig = px.scatter_map(df_nodes, lat="lon", lon="lat", hover_name="education", hover_data=["label", "square"],
                         color=df_nodes['color'],
                         zoom=8,
                         height=1000)
    fig.update_layout(map_style="open-street-map")
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(
        title='Interactive Graph with NetworkX and Plotly',
        hovermode='closest',
    )
    fig.show()

def calculate_distances(geometry: gpd.GeoSeries, point: tuple[float, float]) -> pd.Series:
    dst = gpd.GeoSeries([Point(point), ], crs='epsg:4326').to_crs(epsg=3857)
    return geometry.to_crs(epsg=3857).distance(dst[0])

def add_stops(
        g_: nx.Graph,
        positions: dict[int, tuple[float, float]],
        stops: list[tuple[int, float, float]],
        label: str
):
    geometry = [Point(xy) for xy in list(positions.values())]
    gdf_pos = gpd.GeoDataFrame(list(zip(positions.keys(), positions.values())), columns=['id', 'geometry'], crs='epsg:4326', geometry=geometry)
    for stop_id, lat, lon in stops:
        distances = calculate_distances(gdf_pos.geometry, (lat, lon))
        distances_less = distances[distances < buffer_size]
        if distances_less.shape[0] > 0:
            if g_.has_node(stop_id):
                raise ValueError('The node already exists')
            g_.add_node(stop_id, coordinate=(lat, lon),
                        label=label, education=0, kindergarten=0, school=0, bus_buf=0, tram_stop=0, subway_buf=0,
                        lat=lat, lon=lon)
            for building_id in gdf_pos[distances < buffer_size]['id']:
                g_.add_edge(building_id, stop_id, meters=0)

def build_data():
    cache_file_path = Path('/content') / filename.replace('.csv', '.pkl')
    if cache_file_path.exists():
        with open(cache_file_path, 'rb') as f:
            graph = pickle.load(f)
        return graph
    df = pd.read_csv(filepath)
    data_json = {'features': df.to_dict(orient='records')}
    stops_bus = extract_stops(filepath_bus)
    stops_tram = extract_stops(filepath_tram)
    stops_sub = extract_stops(filepath_subway)
    graph = nx.Graph()
    squares_map = df_squares.set_index('building_id')['building_area'].to_dict()
    n_count = len(data_json['features'])
    for i, building in enumerate(data_json['features']):
        if i % 100 == 0:
            print(f'Calc {i} / {n_count}')
        props = building['properties'] if 'properties' in building else building
        building_id = props['building_id']
        square = float(squares_map[building_id])
        education = int(props['education'] > 0)
        if square < 200:
            print(f'Skip building "{building_id}" with square {square} and education={education}')
            continue
        x_source, y_source = building['x'], building['y']
        graph.add_node(props['building_id'], coordinate=(x_source, y_source),
                       square=square,
                       label='Y' if props['education'] else 'N',
                       kindergarten=props['kindergarten'],
                       school =props['school'],
                       bus_buf=props['bus_buf'],
                       tram_stop=props['tram_stop'],
                       subway_buf=props['subway_buf'],
                       education=education,
                       lat=x_source,
                       lon=y_source
                       )
    print('The nodes was build')
    pos_: dict[int, tuple[float, float]] = nx.get_node_attributes(graph, 'coordinate')
    geometry = [Point(xy) for xy in list(pos_.values())]
    gdf_pos = gpd.GeoDataFrame(list(zip(pos_.keys(), pos_.values())), columns=['id', 'geometry'], crs='epsg:4326', geometry=geometry)
    n_count = len(pos_)
    for i, (building_id, coord) in enumerate(pos_.items()):
        if i % 100 == 0:
            print(f'Calc {i} / {n_count}')
        gdf_pos['distances'] = calculate_distances(gdf_pos.geometry, coord)
        indices = (gdf_pos['distances'] > 0) & (gdf_pos['distances'] < buffer_size)
        if indices.any():
            gdf_close = gdf_pos[gdf_pos['distances'] < buffer_size]
        else:
            gdf_close = gdf_pos.sort_values('distances')[1:n_closest]
        was_added_any = False
        for next_building_id, meters in zip(gdf_close['id'], gdf_close['distances']):
            if next_building_id == building_id:
                continue
            graph.add_edge(building_id, next_building_id, meters=meters)
            was_added_any = True
        if not was_added_any:
            raise ValueError('...')
    print('The graph was build')
    add_stops(graph, pos_, stops_bus, 'bus')
    print('The bus were added')
    add_stops(graph, pos_, stops_tram, 'tram')
    print('The tram were added')
    add_stops(graph, pos_, stops_sub, 'subway')
    print('The subway were added')
    with open(cache_file_path, 'wb') as f:
        pickle.dump(graph, f)
    return graph

if __name__ == '__main__':
    graph_data = build_data()
    draw_plotly(graph_data)


binary classification of graph using graph neural network GCN

In [ ]:
filename = 'data_high1.pkl'

def read_data() -> nx.Graph:
    cache_file_path = Path('/content') / filename
    with open(cache_file_path, 'rb') as f:
        graph = pickle.load(f)
    return graph

G: nx.Graph = read_data()
N_FEATURES = 9

features = []
labels = []
node_mapping = {}
for i, (node_id, node_data) in enumerate(G.nodes(data=True)):
    node_mapping[node_id] = i
    is_stop = 0
    if node_data['label'] == 'bus':
        is_stop = 1
    elif node_data['label'] == 'tram':
        is_stop = 2
    elif node_data['label'] == 'subway':
        is_stop = 3
    features.append([
        node_data.get('square', 1),
        node_data.get('building_area', 1),
        node_data.get('living_area', 1),
        node_data.get('population_balanced', 1),
        node_data['school'],
        node_data['kindergarten'],
        node_data['bus_buf'],
        node_data['tram_stop'],
        node_data['subway_buf'],
    ])
    labels.append(int(node_data['education'] > 0))

features = torch.tensor(features, dtype=torch.float)
num_nodes = features.size(0)
labels = torch.tensor(labels, dtype=torch.long)
train_indices, test_indices = train_test_split(
    np.arange(num_nodes),
    test_size=0.3,
    stratify=labels.numpy(),
    random_state=21
)
edges = list(map(lambda v: (node_mapping[v[0]], node_mapping[v[1]]), G.edges))
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
data = Data(x=features, edge_index=edge_index, y=labels,
            train_mask=train_indices, test_mask=test_indices)
assert edge_index.max() < num_nodes, "Edge index is out of bounds!"

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(N_FEATURES, 32)
        self.bn1 = torch.nn.BatchNorm1d(32)
        self.conv2 = GCNConv(32, 64)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.conv3 = GCNConv(64, 128)
        self.bn3 = torch.nn.BatchNorm1d(128)
        self.conv4 = GCNConv(128, 256)
        self.bn4 = torch.nn.BatchNorm1d(256)
        self.conv5 = GCNConv(256, 512)
        self.bn5 = torch.nn.BatchNorm1d(512)
        self.conv6 = GCNConv(512, 1024)
        self.bn6 = torch.nn.BatchNorm1d(1024)
        self.conv7 = GCNConv(1024, 2)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.conv5(x, edge_index)
        x = self.bn5(x)
        x = F.relu(x)
        x = self.conv6(x, edge_index)
        x = self.bn6(x)
        x = F.relu(x)
        x = self.conv7(x, edge_index)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
class_weights = torch.tensor([1., 10.])
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

def calc_scores(out_, target_) -> tuple[float, float, float]:
    out_max = out_.max(1)[1]
    correct = target_.eq(out_max).sum().item()
    acc = correct / target_.size()[0]
    correct1 = target_[target_.eq(1)].eq(out_max[target_.eq(1)]).sum().item()
    acc1 = correct1 / target_[target_.eq(1)].size()[0]
    f1 = float(f1_score(target_.numpy(), out_max.numpy(), average='macro'))
    return f1, acc, acc1

model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        out = model(data.x, data.edge_index)
        model.train()
        f1_train, acc_train, acc1_train = calc_scores(out[data.train_mask], data.y[data.train_mask])
        f1_test, acc_test, acc1_test = calc_scores(out[data.test_mask], data.y[data.test_mask])
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}, '
              f'f1/train: {f1_train:.2f}, acc/avg/train: {acc_train:.2f}, acc/1/train: {acc1_train:.2f}, '
              f'f1/test: {f1_test:.2f}, acc/avg/test: {acc_test:.2f}, acc/1/test: {acc1_test:.2f}')

def save_to_qgis(data_, pred_):
    data_out = []
    for i, (node_id, node_data) in enumerate(G.nodes(data=True)):
        data_out.append([
            node_id,
            node_data['label'],
            node_data.get('square', 1),
            node_data['kindergarten'],
            node_data['school'],
            node_data['bus_buf'],
            node_data['tram_stop'],
            node_data['subway_buf'],
            node_data['label'],
            node_data['education'],
            node_data['coordinate'][1],
            node_data['coordinate'][0],
            pred_[i]
        ])
    df_out = pd.DataFrame(data_out, columns=['building_id', 'label', 'square', 'kindergarten', 'school', 'bus_buf', 'subway_buf', 'tram_stop',
                                              'label', 'education', 'lat', 'lon', 'predict'])
    print('saving...')
    df_out.iloc[data_.train_mask].to_csv('/content/train2.csv', index=False)
    df_out.iloc[data_.test_mask].to_csv('/content/test2.csv', index=False)

model_filename = 'model.pth'
torch.save(model.state_dict(), model_filename)

model.eval()
with torch.no_grad():
    pred = model(data.x, data.edge_index)
    f1, acc_test, acc1_test = calc_scores(pred[data.test_mask], data.y[data.test_mask])
    save_to_qgis(data, pred.max(1)[1].numpy())
    print(f'TEST: f1: {f1:.2f}, acc/avg/test: {acc_test:.2f}, acc/1/test: {acc1_test:.2f}')